In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.optimize import curve_fit

In [2]:
def ProbDistFuncChi2(chi2, v):
    return ( (chi2**((v/2)-1))*np.exp(-chi2/2) )/( (2**(v/2))*sp.special.gamma(v/2) )

In [3]:
def ProbDistFuncGauss(X, mu, sigma):
    
    return ( np.exp( -((X - mu)**2)/(2*(sigma**2)) ) )/( (sigma*np.sqrt(2*np.pi)) )

In [4]:
Counts = np.array( [9, 48, 142, 154, 438, 521, 405, 318, 299, 100, 57, 9] )
Intervals = np.array( [(-np.inf, -2.5), (-2.5, -2), (-2, -1.5), (-1.5, -1), (-1, -0.5), (-0.5, 0), (0, 0.5), (0.5, 1), (1, 1.5), (1.5, 2), (2, 2.5), (2.5, np.inf)] )
Expected_Values = np.zeros( len(Counts) )
MidVals = np.zeros( len(Counts) )

In [5]:
Total_Counts = Counts.sum()

pon_sum = 0
for i in range(0, len(Counts)):
    
    if i == 0:
        midval = (Intervals[i][1])
        
    elif i == len(Counts) -1:
        midval = (Intervals[i][0])
        
    else:
        midval = (Intervals[i][0] + Intervals[i][1])/2
        
    MidVals[i] = midval
    pon_sum += midval*Counts[i]
    
    
mean = pon_sum/Total_Counts
mean

0.0036

In [6]:
sum_variance = 0

for i in range(0, len(Counts)):
    
    if i == 0:
        midval = (Intervals[i][1])
        
    elif i == len(Counts) -1:
        midval = (Intervals[i][0])
        
    else:
        midval = (Intervals[i][0] + Intervals[i][1])/2
        
    sum_variance += Counts[i]*(midval-mean)**2
    
std = np.sqrt(sum_variance/(Total_Counts-1))
std

1.0153075296440488

In [7]:
popt, pcov = curve_fit( ProbDistFuncGauss, MidVals, Counts, maxfev = 100000 )

C:\Users\santi\anaconda3\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [8]:
mean_sp = popt[0]
std_sp = popt[1]

print(mean_sp)
print(std_sp)

-0.2500398804902577
0.0007646834949014625


In [9]:
for i in range(0, len(Expected_Values)):
    
    area_proportion, error = sp.integrate.quad(ProbDistFuncGauss, Intervals[i][0], Intervals[i][1], args=(mean, std))
    
    Expected_Values[i] = Total_Counts*area_proportion
    
Expected_Values

array([ 17.08588569,  43.47772814, 112.7177174 , 230.37244165,
       371.20775628, 471.60213046, 472.40979697, 373.11829269,
       232.35212451, 114.07628199,  44.1527387 ,  17.4271055 ])

Note que todos los valores esperados, para todos los intervalos, son mayores que 5

In [10]:
A = ((Counts - Expected_Values)**2)/Expected_Values

chi2 = A.sum()
print(chi2)

100.84376491804413


In [11]:
v = Total_Counts - 2
v

2498

In [12]:
chi2_v = chi2/v
chi2_v

0.040369801808664584

De acuerdo a la teoría, un valor de $\chi^2 << 1$ es indicativo de un mal cálculo de las incertidumbres de las mediciones. No obstante, dado que para este ejercicio tomamos $E_i$ como dicha incertidumbre, podemos descartar esta posibiilidad. Por ende, este es un buen fit a los datos.

In [13]:
prob_chi2min, errorprob_chi2min = sp.integrate.quad(ProbDistFuncChi2, chi2, np.inf, v)

print(prob_chi2min)

print(1-sp.stats.chi2.cdf(chi2, v))

C:\Users\santi\AppData\Local\Temp\ipykernel_38836\1557710555.py:2: RuntimeWarning: overflow encountered in scalar power
  return ( (chi2**((v/2)-1))*np.exp(-chi2/2) )/( (2**(v/2))*sp.special.gamma(v/2) )
C:\Users\santi\AppData\Local\Temp\ipykernel_38836\1557710555.py:2: RuntimeWarning: invalid value encountered in scalar divide
  return ( (chi2**((v/2)-1))*np.exp(-chi2/2) )/( (2**(v/2))*sp.special.gamma(v/2) )
C:\Users\santi\AppData\Local\Temp\ipykernel_38836\1557710555.py:2: RuntimeWarning: invalid value encountered in scalar multiply
  return ( (chi2**((v/2)-1))*np.exp(-chi2/2) )/( (2**(v/2))*sp.special.gamma(v/2) )
C:\Users\santi\AppData\Local\Temp\ipykernel_38836\4197301183.py:1: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  prob_chi2min, errorprob_chi2min = sp.integrate.quad(ProbDistFuncChi2, chi2, np.inf, v)


nan
1.0
